# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = r'E:\Documents\2nd-ML100Days\data\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Cabin'] = df['Cabin'].fillna('None')
mean_df = df.groupby('Cabin')['Fare'].mean().reset_index()
df = pd.merge(df, mean_df, on=['Cabin'], how='left')
df = df.drop(['Cabin'], axis=1)
df.head(15)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare_x,Embarked,Fare_y
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q,19.157325
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,51.862500
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,19.157325
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S,19.157325
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C,19.157325


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

6 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare_x', 'Fare_y']



,Pclass,Age,SibSp,Parch,Fare_x,Fare_y
0,3,22.0,1,0,7.2500,19.157325
1,1,38.0,1,0,71.2833,71.283300
2,3,26.0,0,0,7.9250,19.157325
3,1,35.0,1,0,53.1000,53.100000
4,3,35.0,0,0,8.0500,19.157325


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [6]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_org = df.drop(['Fare_y'], axis=1)
train_X = MinMaxScaler().fit_transform(df_org)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7038635542329971


In [7]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MinMaxScaler().fit_transform(df)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7016226402916248
